In [1]:
# # create empty DataFrame df_picks and append row
# my_cols = ['date', 'days_lookback', 'syms_freq', 'symbols']
# # Creating Empty DataFrame and Storing it in variable df_picks
# df_model_top_picks = pd.DataFrame(columns=my_cols)
# df_model_top_picks

In [2]:
def get_trading_date_n(date, idx_close, n, verbose=False):
  """
  This function takes a date, a closing price date index, a number of days (n), and an optional verbosity flag,
  and returns the trading date n days away from the input date if it exists in the index, or None otherwise.

  Args:
    date: The date to start from (as a datetime object).
    idx_close: A dictionary or Series containing date index of close prices.
    n: The number of days to move forward or backward in time (positive for future, negative for past).
    verbose: An optional flag to print additional information about the process (default: False).

  Returns:
    The trading date n days away from the input date if it exists in the index, or None otherwise.
  """

  # Check if the input date is present in the closing price index.
  if date in idx_close:
    # Get the last and current index positions of the input date.
    idx_last = len(idx_close) - 1  # Last index of the closing price index.
    idx_date = idx_close.get_loc(date)  # Index of the input date in the index.

    # Calculate the index of the date n days away from the input date.
    idx_date_n = idx_date + n

    # Print debug information if verbose flag is set.
    if verbose:
      print(f"date: {date} is in idx_close, "
            f"date's position in idx_close is: {idx_date} of {idx_last}, "
            f"n: {n}, idx_date_n: {idx_date_n},")

    # Check if the calculated index is within the bounds of the closing price index.
    if 0 <= idx_date_n <= idx_last:
      # Get the date n days away from the input date using the calculated index.
      date_n = idx_close[idx_date_n]

      # Print debug information if verbose flag is set.
      if verbose:
        print(f"idx_date_n: {idx_date_n} is within bounds of idx_close (0 to {idx_last}), date_n: {date_n}\n")

    else:
      # If the calculated index is out of bounds, set the output date to None.
      date_n = None

      # Print debug information if verbose flag is set.
      if verbose:
        print(f"idx_date_n: {idx_date_n} is out-of-bounds of idx_close (0 to {idx_last})\n")

  else:
    # If the input date is not in the closing price index, set the output date to None.
    date_n = None

    # Print debug information if verbose flag is set.
    if verbose:
      print(f"date: {date} is not in idx_close\n")

  # Return the date n days away from the input date if it exists in the index, or None otherwise.
  return date_n

In [3]:
def get_portf_buy(df_close, date, str_symbols, portf_target, verbose=False):
  """
  This function takes a date, a string of symbols (separated by commas), and a target portfolio value,
  and calculates the purchase amounts for each symbol based on equal dollar allocation.

  Args:
    df_close: A Pandas DataFrame with stock closing prices indexed by date and symbol.
    date: The date for which to calculate the portfolio purchase.
    str_symbols: A string containing comma-separated stock symbols.
    portf_target: The desired total value of the portfolio.
    verbose: Whether to print detailed information about the calculations (default: False).

  Returns:
    A tuple containing:
      - date: The date for the calculated portfolio.
      - l_syms: A list of symbols purchased for the portfolio.
      - ar_price: An array of closing prices for the purchased symbols.
      - ar_sym_share: An array of the number of shares purchased for each symbol.
      - ar_sym_amt: An array of the actual dollar amount invested in each symbol.
      - portf_value: The total actual value of the purchased portfolio.
  """

  import numpy as np
  from ast import literal_eval

  # Convert the comma-separated string of symbols into a list.
  l_syms = literal_eval(str_symbols)

  # Extract an array of closing prices for the specified symbols on the given date.
  ar_price = df_close.loc[date][l_syms].values

  # Calculate the number of symbols in the portfolio.
  sym_cnt = len(l_syms)

  # Calculate the target dollar investment per symbol based on equal allocation.
  amt_per_sym = portf_target / sym_cnt

  # Calculate the number of shares to purchase for each symbol (rounded down to whole shares).
  ar_sym_share = np.floor(amt_per_sym / ar_price)

  # Calculate the actual dollar amount invested in each symbol.
  ar_sym_amt = ar_price * ar_sym_share

  # Calculate the total actual portfolio value.
  portf_value = sum(ar_sym_amt)

  if verbose:
    # Print detailed information about the calculations.
    # NOTE: The f-string formatting options are used for nicer printing.
    print(f'{date = }, {l_syms = }, {ar_price = }, {ar_sym_share = }, {ar_sym_amt = }, {portf_value = }')
    print(f'{date} {portf_value = }')

  # Return the calculated portfolio information.
  return date, l_syms, ar_price, ar_sym_share, ar_sym_amt, portf_value



In [4]:
def get_SPY_buy(df_close, date, portf_target, symbol="SPY", verbose=False):
  """
  This function calculates the number of SPY shares to buy and the total investment amount
  based on a target portfolio value and closing price on a specific date.

  Args:
    df_close: A pandas DataFrame containing closing prices for various symbols.
    date: The date for which to calculate the SPY buy (as a string or datetime object).
    portf_target: The desired total value of the investment.
    symbol: The symbol of the security to buy (default: "SPY").
    verbose: An optional flag to print additional information (default: False).

  Returns:
    A tuple containing the following:
      * date: The date for which the calculation was made.
      * price: The closing price of the specified symbol on the given date.
      * share: The number of shares to buy (rounded down to the nearest whole number).
      * value: The total investment amount in the specified symbol.
  """

  # Import NumPy for array operations.
  import numpy as np

  # Extract the closing price of the specified symbol on the given date.
  price = df_close.loc[date][symbol]

  # Calculate the number of shares to buy by dividing the target portfolio value by the price,
  # rounded down to the nearest whole number.
  share = np.floor(portf_target / price)

  # Calculate the total investment amount by multiplying the number of shares by the price.
  value = price * share

  # Print detailed information if verbose flag is set.
  if verbose:
    # Print individual components with formatting (using f-strings) for clarity.
    print(f'{date = }, {symbol = }, {price = }, {share = }, {value = }')
    # Alternatively, you can print a more concise message:
    print(f'{date} {symbol} {value = }')

  # Return the calculated values as a tuple.
  return date, price, share, value

In [5]:
def any_not_in_list(list1, list2):
  """
  Checks if any items in list1 are not in list2.

  Args:
    list1: A list of items.
    list2: Another list of items.

  Returns:
    True if any item in list1 is not in list2, False if all are present.
  """
  return bool(set(list1) - set(list2))

In [6]:
import pandas as pd
import numpy as np
# from itertools import product
from ast import literal_eval
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.width', 900)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# fp_df_picks  = f'df_picks'  # stock picks by criteria: CAGR/UI, CAGR/rtn_std, rtd/UI
fp_df_model_top_picks = 'df_model_top_picks'  # top stock picks from model developed by back test
fp_df_close_clean = 'df_close_clean'  # historic stocks' closing price

verbose = True
# verbose = False

#### Load df_close, symbols' closing price and df_picks_mp, model's top picks

In [7]:
df_close = pickle_load(path_data_dump, fp_df_close_clean)
df_picks_mp = pickle_load(path_data_dump, fp_df_model_top_picks)

#### Get the start and end dates of df_picks_mp

In [8]:
dates_sorted = sorted(df_picks_mp.date.tolist())
date_start_picks_mp = dates_sorted[0]
date_end_picks_mp = dates_sorted[-1]
print(f'date_start_picks_mp: {date_start_picks_mp}\ndate_end_picks_mp:   {date_end_picks_mp}') 

date_start_picks_mp: 2023-03-15
date_end_picks_mp:   2023-12-15


#### Select date range in df_close to match df_picks_mp

In [9]:
# Create a boolean mask for rows between date1 and date2 (inclusive)
mask = (df_close.index >= date_start_picks_mp) & (df_close.index <= date_end_picks_mp)

# Select rows using the mask
df_close = df_close.loc[mask]

# Trading dates
idx_close = df_close.index.strftime('%Y-%m-%d')

# Symbols in df_close
symbols_df_close = df_close.columns  # symbols in df_close

df_close

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABR,ABT,ACGL,ACHC,ACIW,ACLS,ACRS,ADBE,...,XRX,XYL,YELP,YUM,YUMC,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-03-15,133.288239,39.372566,13.86,119.255257,152.371918,32.977055,149.485596,10.526908,96.316017,62.740002,68.459999,22.700001,123.860001,7.990,333.609985,...,14.261086,95.604477,29.170000,125.356514,59.979794,26.028101,124.543419,287.739990,73.779999,39.919998,29.475050,25.606165,162.472290,19.500000,20.806547
2023-03-16,136.062805,40.342766,14.12,119.509888,155.220352,33.193268,150.688766,10.499517,97.527351,66.070000,70.230003,25.780001,129.960007,8.120,353.290009,...,14.451233,96.208496,29.780001,126.273537,60.515686,26.922306,124.712440,294.929993,74.510002,40.849998,30.818754,27.639652,165.193893,19.250000,20.796629
2023-03-17,132.442932,38.758762,13.98,116.934204,154.373779,32.883053,149.640839,10.216486,95.538002,63.150002,69.459999,26.940001,128.220001,7.890,358.140015,...,13.899805,93.802299,29.330000,125.080414,60.257668,27.720362,123.598854,288.709991,74.129997,39.810001,28.736017,28.083862,163.366241,18.540001,20.360270
2023-03-20,133.795395,39.877464,13.96,116.229065,156.764084,33.024059,151.484421,10.791677,96.384964,65.610001,70.449997,26.150000,131.509995,7.950,362.880005,...,14.118475,96.693703,29.690001,127.082108,60.485916,28.028044,125.607292,290.839996,74.650002,39.830002,28.966366,27.659395,164.707199,18.110001,20.657787
2023-03-21,136.251740,41.491177,14.37,117.316147,158.636505,33.334270,152.115128,10.873848,96.837975,67.209999,70.959999,26.200001,133.449997,8.240,374.220001,...,14.546309,97.555176,30.059999,126.756706,60.704235,27.489597,127.058952,294.429993,75.570000,41.939999,30.991514,27.866692,165.124374,18.549999,21.004896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-11,128.970001,24.930000,13.72,57.080002,193.179993,29.820000,151.240005,14.100000,106.220001,79.610001,74.959999,28.000000,125.580002,0.973,625.200012,...,15.170000,107.699997,44.099998,126.629997,39.919998,36.939999,117.980003,239.649994,63.730000,43.820000,38.630001,21.020000,189.460007,18.969999,29.610001
2023-12-12,128.789993,24.049999,14.04,56.250000,194.710007,30.559999,153.240005,14.190000,106.680000,80.279999,75.910004,28.320000,126.190002,1.010,633.659973,...,15.710000,107.940002,44.480000,128.070007,40.320000,39.029999,117.449997,239.380005,62.560001,45.419998,38.130001,21.020000,191.470001,18.570000,29.230000
2023-12-13,133.740005,25.990000,14.09,60.830002,197.960007,30.809999,154.300003,14.880000,107.250000,78.379997,78.959999,29.240000,131.500000,1.090,624.260010,...,16.709999,109.180000,44.490002,130.929993,39.750000,38.500000,117.900002,244.429993,64.720001,50.279999,41.820000,20.400000,197.410004,19.670000,29.320000


In [10]:
date_buy = []  # buy date of portfolio
shares_syms = []  # lists of shares of each symbol brought on date_buy
value_portf = [] # list of porfolio value on date_buy
shares_SPY = []  # list of shares of SPY brought on date_buy
value_SPY = []  # list of value of SPY shares on date_buy 

# Loop through each date and its associated symbols
for date, syms in zip(df_picks_mp.date, df_picks_mp.symbols):
  # Calculate the date n days away (buy date)
  n = 1  # Set n=1 for buy date
# n = 4  # n=4 is the sell date  

  next_date_n = get_trading_date_n(date, idx_close, n, verbose=verbose)

  # Check if the buy date is within the DataFrame's date range
  if next_date_n in idx_close:
    close_date_n = next_date_n
    print(f'date: {date}, n: {n}, next_date_n: {next_date_n} is in df_close')
    print(f'picks for {date}: {syms}')
  else:
    close_date_n = None
    print(f'date: {date}, n: {n}, next_date_n: {next_date_n} is not in df_close')
    print(f'picks for {date}: {syms}')

  # Convert list stored as string back to list
  l_syms = eval(syms)  # Use `eval` instead of `literal_eval`

  # Check if any symbol(s) in the picks are not in df_close
  sym_not_in_df_close = any(symbol not in df_close.columns for symbol in l_syms)

  # If buy date or symbols are unavailable, set portfolio values to None
  if close_date_n is None or sym_not_in_df_close:
    date_buy.append(None)
    shares_syms.append(None)
    value_portf.append(None)
    shares_SPY.append(None)
    value_SPY.append(None)
    print(f"No data for next_date_n {next_date_n}, pick's portf value = None")
    print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

  # Otherwise, calculate portfolio and SPY values for the buy date
  else:
    p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
      get_portf_buy(df_close, next_date_n, syms, portf_target=1000, verbose=verbose)
    s_date, s_price, s_share, s_value = get_SPY_buy(df_close, next_date_n, portf_target=1000, verbose=verbose)

    date_buy.append(p_date)
    shares_syms.append(p_ar_sym_share)
    value_portf.append(p_portf_value)
    shares_SPY.append(s_share)
    value_SPY.append(s_value)

    print(f"next_date_n pick's portf value = {p_portf_value}")
    print(f'next_date_n SPY portf value =    {s_value}')

  print('='*20, '\n')

date: 2023-12-15 is in idx_close, date's position in idx_close is: 191 of 191, n: 1, idx_date_n: 192,
idx_date_n: 192 is out-of-bounds of idx_close (0 to 191)

date: 2023-12-15, n: 1, next_date_n: None is not in df_close
picks for 2023-12-15: ['SHV']


NameError: name 'date_buy' is not defined

In [ ]:
# import numpy as np
# # from ast import literal_eval
# z_date_syms = zip(df_picks_mp.date, df_picks_mp.symbols)

# idx_close = df_close.index.strftime('%Y-%m-%d')

# n = 1  # n=1 is the buy date
# # n = 4  # n=4 is the sell date

# date_buy = []  # buy date of portfolio
# shares_syms = []  # lists of shares of each symbol brought on date_buy
# value_portf = [] # list of porfolio value on date_buy
# shares_SPY = []  # list of shares of SPY brought on date_buy
# value_SPY = []  # list of value of SPY shares on date_buy 
# for date, syms in z_date_syms:
#     next_date_n = get_trading_date_n(date, idx_close, n, verbose=verbose)

#     if next_date_n in idx_close:
#         close_date_n = next_date_n
#         print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is in df_close')
#         print(f'picks for {date}: {syms}')      
#     else:
#         close_date_n = None
#         print(f'date: {date},  n: {n},  next_date_n: {next_date_n} is not in df_close')
#         print(f'picks for {date}: {syms}')        

#     if close_date_n is None:
#         p_date = None  # portf. buy date
#         p_l_syms = None  # portf. list of symbols
#         p_ar_price = None  # portf. list of symbols' close
#         p_ar_sym_share = None  # portf. list of symbols' share
#         p_ar_sym_amt = None  # portf. list of symbols' dollar value 
#         p_portf_value = None  # portf. value is none, data not available in df_close
        
#         s_date = None  # SPY buy date
#         s_price = None # SPY list of symbols' close
#         s_share = None # SPY list of symbols' share
#         s_value = None  # SPY value is none, data not available in df_close

#         date_buy.append(p_date)
#         shares_syms.append(p_l_syms)
#         value_portf.append(p_portf_value)
#         shares_SPY.append(s_share)
#         value_SPY.append(s_value)    
        
#         print(f"No data for next_date_n {next_date_n}, pick's portf value = None")
#         print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

#     else:    
#         p_date, p_l_syms, p_ar_price, p_ar_sym_share, p_ar_sym_amt, p_portf_value = \
#             get_portf_buy(df_close, next_date_n, syms, portf_target=1000, verbose=verbose)
#         s_date, s_price, s_share, s_value = \
#             get_SPY_buy(df_close, next_date_n, portf_target=1000, verbose=verbose)
                
#         date_buy.append(p_date)
#         shares_syms.append(p_ar_sym_share)
#         value_portf.append(p_portf_value)
#         shares_SPY.append(s_share)
#         value_SPY.append(s_value)

#         print(f"next_date_n pick's portf value = {p_portf_value}")
#         print(f'next_date_n SPY portf value =    {s_value}')

#     print('='*20, '\n')       
    

#### Below cells calculate past performance of model picks

In [ ]:
def calc_portf_shares(df_close, date, str_symbols, portf_target):
  # calculate number of shares to buy for symbols in str_symbols to meet port_target
  import numpy as np
  from ast import literal_eval    
  l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
  # array of closing prices corresponding to symbols in l_syms    
  ar_price = df_close.loc[date][l_syms].values  
  sym_cnt = len(l_syms)  # number of symbols
  amt_per_sym = portf_target / sym_cnt  # target dollar investment in each symbol
  ar_shares = np.floor(amt_per_sym / ar_price)  # array of shares for each symbol
  return ar_shares

In [ ]:
def calc_portf_value(df_close, date, str_symbols, ar_shares, verbose=False):
    import numpy as np
    from ast import literal_eval    
    l_syms = literal_eval(str_symbols)  # convert list stored as str back to list
    # array of closing prices corresponding to symbols in l_syms    
    ar_price = df_close.loc[date][l_syms].values  
    ar_value = ar_price * ar_shares  # array of actual dollar amount invested in each symbol
    portf_value = sum(ar_value)  # total actual portfolio value
    if verbose:
        print(f'{date = }, {l_syms = }, {ar_price = }, {ar_shares = }, {ar_value = }, {portf_value = }')            
        print(f'{date} {portf_value = }')
    return date, l_syms, ar_price, ar_shares, ar_value, portf_value

In [ ]:
def get_next_date_n(df_close, date, n=1):

    """Get the next n(default=1) date from df_close. date has to in df_close.index.

    Edge case: The function won't return the first date in df_close as the
    next trading date, if date is a trading date just before the first date in df_close.
    """

    len_df_close = len(df_close)
    idx_dates = df_close.index.strftime('%Y-%m-%d')  # date index of df_close    

    if date in idx_dates:
        idx_next_date = idx_dates.get_loc(date) + n  # get index position of next date
        print(f'n: {n},  idx_next_date: {idx_next_date}')

        if idx_next_date >= 0:  # if idx_nex_date is negative, stop index loop back from index[0] to index[-1]    
            if idx_next_date >= (len_df_close - n):  # next date's index is within bound of df_close
                next_date = idx_dates[idx_dates.get_loc(date) + n]  # get the next date
                return next_date
        
              
            else:  # idx_next_date is out-of_bounds df_close
                print(f"{date} is the last date in df_close. No data for next {n} day")
                return None    
        
        
        else:  # idx_next_date is negative, no data in df_close
            # no data from df_close
            print(f'idx_next_date: {idx_next_date} is negative, no data for {n} days from {date} in df_close')

    else:     
        # no data from df_close
        print(f'no data for {date} in df_close')
        return None

In [ ]:
def is_date_in_close(date, df_close):
  idx_close = df_close.index.strftime('%Y-%m-%d')
  if date in idx_close:
    return date
  else:
    return None

In [ ]:
def calc_portf_value_date_buy_(dates_in_days_lookbacks, my_symbols, df_close, portf_target, n, verbose=False):
  
  z_date_syms = zip(dates_in_days_lookbacks, my_symbols)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  shares_SPY = []  # list of shares of SPY brought on date
  value_SPY = []  # list of value of SPY shares on date 

  for date, syms in z_date_syms:
    next_date_n = get_trading_date_n(date, idx_close, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_close)



    l_syms = literal_eval(syms)  # convert list stored as str back to list
    # True if symbol(s) in l_syms is not a column in df_close
    sym_not_in_df_close = any_not_in_list(l_syms, symbols_df_close)

    if close_date_n is None or sym_not_in_df_close:



    # if close_date_n is None:
      p_date = None
      p_ar_shares = None
      p_portf_value = None  # set to None when data are not available in df_close
      SPY_shares = None
      SPY_value = None  # set to None when data are not available in df_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        print(f'No data for close_date_n {close_date_n}, SPY portf value =    None')

    else:    
      p_ar_shares = calc_portf_shares(df_close, close_date_n, syms, portf_target)
      p_date, l_syms, ar_price, ar_shares, ar_value, p_portf_value = \
        calc_portf_value(df_close, close_date_n, syms, p_ar_shares, verbose)

      syms = str(['SPY'])
      SPY_shares = calc_portf_shares(df_close, close_date_n, syms, portf_target)
      date, l_syms, ar_price, ar_shares, ar_value, SPY_value = \
        calc_portf_value(df_close, close_date_n, syms, SPY_shares, verbose)

      if verbose:
        print(f"close_date_n pick's portf value = {p_portf_value}")
        print(f'close_date_n SPY portf value =    {SPY_value}')

    date_exec.append(p_date)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_portf_value)
    shares_SPY.append(SPY_shares)
    value_SPY.append(SPY_value)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf, shares_SPY, value_SPY       
    

In [ ]:
def calc_portf_value_date_buy(dates_in_days_lookbacks, my_symbols, df_close, portf_target, n, verbose=False):
  
  z_date_syms = zip(dates_in_days_lookbacks, my_symbols)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  shares_SPY = []  # list of shares of SPY brought on date
  value_SPY = []  # list of value of SPY shares on date 

  for date, syms in z_date_syms:
    next_date_n = get_trading_date_n(date, idx_close, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_close)

    if close_date_n is None:
      p_date = None
      p_ar_shares = None
      p_portf_value = None  # set to None when data are not available in df_close
      SPY_shares = None
      SPY_value = None  # set to None when data are not available in df_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        print(f'No data for close_date_n {close_date_n}, SPY portf value =    None')

    else:    
      p_ar_shares = calc_portf_shares(df_close, close_date_n, syms, portf_target)
      p_date, l_syms, ar_price, ar_shares, ar_value, p_portf_value = \
        calc_portf_value(df_close, close_date_n, syms, p_ar_shares, verbose)

      syms = str(['SPY'])
      SPY_shares = calc_portf_shares(df_close, close_date_n, syms, portf_target)
      date, l_syms, ar_price, ar_shares, ar_value, SPY_value = \
        calc_portf_value(df_close, close_date_n, syms, SPY_shares, verbose)

      if verbose:
        print(f"close_date_n pick's portf value = {p_portf_value}")
        print(f'close_date_n SPY portf value =    {SPY_value}')

    date_exec.append(p_date)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_portf_value)
    shares_SPY.append(SPY_shares)
    value_SPY.append(SPY_value)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf, shares_SPY, value_SPY       
    

In [ ]:
def calc_portf_value_date_n(dates_in_days_lookbacks, my_symbols, df_close, my_portf_shares, my_SPY_shares, n, verbose=False):
  
  z_date_syms_shares = zip(dates_in_days_lookbacks, my_symbols, my_portf_shares, my_SPY_shares)

  date_exec = []  # buy date of portfolio
  shares_syms = []  # lists of shares of each symbol brought on date
  value_portf = [] # list of porfolio value on date
  shares_SPY = []  # list of shares of SPY brought on date
  value_SPY = []  # list of value of SPY shares on date 

  for date, symbols, portf_shares, SPY_shares in z_date_syms_shares:
    next_date_n = get_trading_date_n(date, idx_close, n, verbose=False)
    close_date_n = is_date_in_close(next_date_n, df_close)

    if close_date_n is None:
      p_date_exec = None
      p_ar_shares = None
      p_value_portf = None  # set to None when data are not available in df_close
      SPY_ar_shares = None
      SPY_value_portf = None # set to None when data are not available in df_close

      if verbose:
        print(f"No data for close_date_n {close_date_n}, pick's portf value = None")
        # print(f'No data for next_date_n {next_date_n}, SPY portf value =    None')

    else: 
      if portf_shares is None:
        p_date_exec = None
        p_ar_shares = None
        p_value_portf = None
        SPY_ar_shares = None
        SPY_value_portf = None

      else:  
        p_date_exec, p_ar_syms, p_ar_price, p_ar_shares, p_ar_value, p_value_portf = \
          calc_portf_value(df_close, close_date_n, symbols, portf_shares, verbose)
        
        SPY = str(['SPY'])
        SPY_date_exec, SPY_ar_syms, SPY_ar_price, SPY_ar_shares, SPY_ar_value, SPY_value_portf = \
          calc_portf_value(df_close, close_date_n, SPY, SPY_shares, verbose) 

        if verbose:
          print(f"next_date_n pick's portf value = {p_value_portf}")
          print(f'next_date_n SPY portf value =    {SPY_value_portf}')

    date_exec.append(p_date_exec)
    shares_syms.append(p_ar_shares)
    value_portf.append(p_value_portf)
    shares_SPY.append(SPY_ar_shares)
    value_SPY.append(SPY_value_portf)

    print('='*20, '\n')

  return date_exec, shares_syms, value_portf, shares_SPY, value_SPY  

In [ ]:
df_picks_mp

In [ ]:
df_picks_mp.date

In [ ]:
df_picks_mp.symbols

In [ ]:
df_close

In [ ]:
# total target investment for each day's picks, if day's pick has 2 symbols, the function will try to invest $500 for each symbol
portf_target = 1000  
date_buy, shares_syms, value_portf, shares_SPY, value_SPY = \
  calc_portf_value_date_buy_(df_picks_mp.date, df_picks_mp.symbols, df_close, portf_target, n=1, verbose=verbose)

In [ ]:
# # total target investment for each day's picks, if day's pick has 2 symbols, the function will try to invest $500 for each symbol
# portf_target = 1000  
# date_buy, shares_syms, value_portf, shares_SPY, value_SPY = \
#   calc_portf_value_date_buy(df_picks_mp.date, df_picks_mp.symbols, df_close, portf_target, n=1, verbose=verbose)

In [ ]:
df_picks_mp['date_buy'] = date_buy
df_picks_mp['sh_portf_buy'] = shares_syms
df_picks_mp['$_portf_buy'] = value_portf
df_picks_mp['sh_SPY_buy'] = shares_SPY
df_picks_mp['$_SPY_buy'] = value_SPY
# df_picks_mp.tail()
# df_picks_mp.head()
df_picks_mp

In [ ]:
date_exec, shares_syms, value_portf, shares_SPY, value_SPY  = \
  calc_portf_value_date_n(df_picks_mp.date, df_picks_mp.symbols, df_close, df_picks_mp.sh_portf_buy, df_picks_mp.sh_SPY_buy, n=4, verbose=verbose)

In [ ]:
df_picks_mp['date_sell'] = date_exec
df_picks_mp['sh_portf_sell'] = shares_syms
df_picks_mp['$_portf_sell'] = value_portf
df_picks_mp['%_portf_chg'] = (df_picks_mp['$_portf_sell'] / df_picks_mp['$_portf_buy'] - 1) * 100

df_picks_mp['sh_SPY_sell'] = shares_SPY
df_picks_mp['$_SPY_sell'] = value_SPY
df_picks_mp['%_SPY_chg'] = (df_picks_mp['$_SPY_sell'] / df_picks_mp['$_SPY_buy'] - 1) * 100

df_picks_mp['dif_%_chg'] = df_picks_mp['%_portf_chg'] - df_picks_mp['%_SPY_chg']

In [ ]:
# # get symbol's close price for that date
# date = '2023-03-16'
# # _date = '2023-03-21'
# # _sym = 'GE'
# # _sym = 'NVDA'
# _sym = 'FTSM'
# # _sym = 'SHV'
# _sym = 'SPY'
# # _c = df_close.loc['2023-04-11']['NVDA']
# _c = df_close.loc[_date][_sym]
# _c

#### dif_%_chg is the percentage change of model icks - percentage change of SPY

In [ ]:
df_picks_mp.head(20)

In [ ]:
df_picks_mp['win'] = np.where(df_picks_mp['dif_%_chg']>0,1,0)
df_picks_mp.head(20)

### Model Performance Not Good

In [ ]:
wins = df_picks_mp['win'].sum()
attempts = len(df_picks_mp['dif_%_chg'].dropna())

win_rate = wins / attempts
print(f'win_rate: {win_rate:0.6f}, wins: {wins}, attempts: {attempts}')
print(f'sum(df_picksf_%_chg): {df_picks_mp["dif_%_chg"].sum():0.6f}')


In [ ]:
df_picks = df_picks_mp.copy()

In [ ]:
df_modelpicks = df_picks[df_picks['dif_%_chg'].isnull()].copy()
pickle_dump(df_modelpicks, path_data_dump, 'df_modelpicks')
_df_picks = pickle_load(path_data_dump, 'df_modelpicks')
_df_picks

In [ ]:
idx_last_row = df_modelpicks.index[-1]
df_modelpicks_results = df_picks.iloc[(idx_last_row + 1)::].copy()
pickle_dump(df_modelpicks_results, path_data_dump, 'df_modelpicks_results')
_df_picks = pickle_load(path_data_dump, 'df_modelpicks_results')
_df_picks